In [64]:
import json

# Open and read the JSON file
with open('a-aal-MOT-INV-BIL1-2017-07-05-anlc-comp.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [65]:
with open('a-aal-MOT-INV-BIL1-2017-07-05-anlc-compEd-.json', 'r', encoding='utf-8') as file:
    data2 = json.load(file)

In [66]:
SINGULARS = ("lo", "me", "te", "la", "le", "vos")
PLURALS = ("los", "las", "les", "nos")
BOTH = SINGULARS + PLURALS

In [107]:
ids_to_save = {}
words_saved = []
for info in data:
    ID = info['id']
    for sentences in info["sentences"]:
        for word in sentences:
            if word['text'].endswith(BOTH) and word['text'] not in words_saved:
                try:
                    ids_to_save[ID] = {'word': word['text'], 'tag': word['upos']}
                    words_saved.append(word['text'])
                except KeyError:
                    ids_to_save[ID] = {'word': word['text']}
                    words_saved.append(word['text'])
                    print(word)
                    print(ID)



print(ids_to_save)
print(words_saved)

{'id': [10, 11], 'text': 'traerlo', 'start_char': 45, 'end_char': 52}
156
{'id': [2, 3], 'text': 'terminarlo', 'start_char': 7, 'end_char': 17, 'misc': 'SpaceAfter=No'}
188
{'id': [3, 4], 'text': 'hacerlo', 'start_char': 45, 'end_char': 52, 'misc': 'SpaceAfter=No'}
265
{'id': [2, 3], 'text': 'darle', 'start_char': 6, 'end_char': 11}
543
{'id': [3, 4], 'text': 'buscarlo', 'start_char': 8, 'end_char': 16, 'misc': 'SpaceAfter=No'}
677
{24: {'word': 'cumple', 'tag': 'VERB'}, 28: {'word': 'vos', 'tag': 'PRON'}, 29: {'word': 'naciste', 'tag': 'VERB'}, 32: {'word': 'dijiste', 'tag': 'VERB'}, 34: {'word': 'este', 'tag': 'DET'}, 38: {'word': 'le', 'tag': 'PRON'}, 43: {'word': 'las', 'tag': 'DET'}, 45: {'word': 'menos', 'tag': 'ADV'}, 50: {'word': 'lo', 'tag': 'PRON'}, 55: {'word': 'la', 'tag': 'DET'}, 62: {'word': 'tranquilo', 'tag': 'ADJ'}, 64: {'word': 'los', 'tag': 'DET'}, 70: {'word': 'Buenos', 'tag': 'PROPN'}, 76: {'word': 'hermanos', 'tag': 'NOUN'}, 80: {'word': 'sólo', 'tag': 'ADV'}, 125

In [112]:
ALL_words = []
ALL_tags = []

for info in data2:
    ID = info['id']
    sent = info["text"]
    if ID not in ids_to_save.keys():
        continue

    for sentences in info["sentences"]:
        for word in sentences:
            looking_for = ids_to_save[ID]['word']

            if word['text'].endswith(BOTH) and word['text'] == ids_to_save[ID]['word']:

                ALL_words.append(word['text'])
                if type(word['id']) != list:
                    if word['upos'] == ids_to_save[ID]['tag']:
                        sent = None
                    if 'tag' in ids_to_save[ID]:
                        ALL_tags.append((ids_to_save[ID]['tag'], word['upos'], ID, None, sent))
                    else:
                        ALL_tags.append((None, word['upos'], ID, None, sent))
                elif type(word['id']) == list:
                    split_upos = ''
                    split_words = ''
                    for id in word['id']:
                        for word2 in sentences:
                            if id == word2['id']:
                                split_upos += word2['upos'] + ", "
                                split_words += word2['text'] + ", "
                        split_upos = split_upos[:-1]
                        split_words = split_words[:-1]
                    if 'tag' in ids_to_save[ID]:
                        ALL_tags.append((ids_to_save[ID]['tag'], split_upos, ID, split_words, sent))
                    else:
                        ALL_tags.append((None, split_upos, ID, split_words, sent))

pairings = (list(set(zip(ALL_words, ALL_tags))))


In [113]:
import csv
with open("output_pairings.csv", mode="w", newline="", encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['words', "OG", "Edited", "SentID", "Split Words", "Sentence"])
    for x, (a, b, c, d, e) in pairings:
        writer.writerow([x, a, b, c, d, e])
